# Basketball challenge

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import serialize
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


results_files_name = serialize.get_all_results_names()
results_files_name.reverse()
out = widgets.Output()

w = widgets.Dropdown(
    options=results_files_name,
    description='Result',
)
def update(filename):
    data_from_csv = pd.read_csv(filename)
    f,ax1 = plt.subplots(figsize =(15,10))
    plt.title('Train/Test Performance',fontsize = 20,color='blue')
    #plt.text(0,0,'Train',color='blue',fontsize = 18)
    #plt.text(0,0,'Test',color='red',fontsize = 18)
    sns.pointplot(x='epochs',y='trainloss', data=data_from_csv)
    ax = sns.pointplot(x='epochs',y='testloss', color='red',data=data_from_csv)
    for x,y,correct,total in zip(data_from_csv['epochs'],data_from_csv['testloss'],data_from_csv['correct'],data_from_csv['test']):
        #change f'{z:.2f}' to str(z) if you want something simpler
        ax.text(x-1,y+0.01,f'{(correct/total)*100:.2f}%',horizontalalignment='center',color='red',weight='light')
    plt.xlabel('Epocs',fontsize = 15,color='blue')
    plt.ylabel('Loss',fontsize = 15,color='blue')
    display(w)
    out

def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        clear_output()
        update(change['new'])

w.observe(on_change)
update(results_files_name[0])


# Variables

In [ ]:
%reload_ext autoreload
%autoreload 2
import Basketball

data = 'data'
data48x48 = 'cache'
num_frames = 96
split = 'training'
EPOCHS = 50
optics = 'optics'
dp = Basketball.Basketball(data, width=128, height=128,  num_frames=num_frames, split=split)

In [ ]:
#dp.run('FFNN', testeverytrain=True, EPOCHS=EPOCHS)
#dp.run('CNN3D', testeverytrain=True, EPOCHS=EPOCHS)
dp.run('CNN2DLSTM', testeverytrain=True, EPOCHS=EPOCHS)
#dp.run('OPTICALCONV3D', testeverytrain=True, EPOCHS=EPOCHS, opticalpath=of_save_path)

In [ ]:
#dp.run('FFNN', testeverytrain=True, EPOCHS=EPOCHS)
#dp.run('CNN3D', testeverytrain=True, EPOCHS=EPOCHS)
#dp.run('CNN2DLSTM', testeverytrain=True, EPOCHS=EPOCHS, lr=0.001, ADAm, momentum=0.9)
dp.run('OPTICALCONV3D', testeverytrain=True, EPOCHS=EPOCHS)

# Plot 

In [ ]:
#%matplotlib inline
#from IPython import display
#import torchvision.transforms.functional as F
#for batch, target in trainset:
#    print("here is the ",target)
#    view1 = batch[0]
#    for idx, img in enumerate(view1):
#        img1 = F.to_pil_image(img)
#        display.clear_output(wait=True)
#        display.display(img1.resize((320,240)), Image.NEAREST)
#    break

# Display Video

In [ ]:
#from IPython import display
#import torchvision.transforms.functional as F
#sets = DataLoader(dataset)
#for bath, target in sets:
#    print("here is the ",target)
#    view1 = batch[0][0]
#    for idx, img in enumerate(view1):
#        img1 = F.to_pil_image(img)
#        display.display(img1.resize((320,240)), Image.NEAREST)
#        display.clear_output(wait=True)
#    break

# Models

Initialize weights
image preprocessing
normalizing the inputs
regularizations

inceptio v4
inceptionv4
resnet 101
resnet 152

1x1 convolution

efficient architectures
How do we obtain networks that are computationally and memory efficient
enough to run on mobile devices, possibly with limited power supply?

Width multiplier:
Reduce the number of channels in each layer. This has been
called width multiplier in „MobileNets: Efficient Convolutional Neural Networks
for Mobile Vision Applications“.
What is the effect on the computational complexity and on the parameters?

resolution multiplier 
Reduce the resolution of the input image. This has been called
resolution multiplier in „MobileNets: Efficient Convolutional Neural Networks for
Mobile Vision Applications“.




Direct approach
Model based approach
Model and learning-based approach

https://paperswithcode.com/paper/quo-vadis-action-recognition-a-new-model-and

1. [Implemented] Feed Forward Neural Network (FFNN)
View 1 and View 2 are concatenated side by side at first and each of the samples are then also concatenated. In this way a vector 
is formed which is then feed to the Feed Forward Neural Network(FFNN). Other variation could be going deep with multiple hidden layers
2. CNN2  [Implemented]
    view1->conv3d
    view2->conv3d-------------> these are somehow combined and 
    optical(view1) -> conv3d--> flatted to predict 2 classes
    optical(view2) -> conv3d
3. CNN3 [Implemented] (use LTSM) -> https://github.com/HHTseng/video-classification
    encode each frame with conv2d and send the output to be decoded to a lstm
4. CNN4 [Implemented]
    view1 and view2 and concatinated side by side and feed through the conv3d and flattened feed through linear layer

Regularization: Adding a term to a loss function that penalizes for high weights. It tradies in some of the ability to fit the training data well for the ability to have a model generalize better to the data it has not seen before.
E(Theta) = Sum of all inputs((target - input) ** 2 ) + (lamda/2*m) * weights**2

Generalization:
    It is a technique to ensure that architecture as well as the training data are chosen in such as way that the network makes a good prediction during training.
    1. Validate: Without validation it is impossible to judge whether my model is reasonable or starts overfitting the data. 
    2. Agumentation.
    3. Ensemble learning
    4. Dropout
    5. Regularization with a penalty
    6. Data augmentation
    7. Early stopping